In [1]:
# 필요한 라이브러리를 가져오자.
# numpy, pandas
import numpy as np
import pandas as pd

In [2]:
import datetime
def make_datetime(row):
    return datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S.%f')

In [3]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [4]:
df = pd.read_csv('lookback_data.csv')

In [5]:
df.head()

,adid,datetime,partner_key,event_name,amount,partner_index
0,0000a0276a0870de648c427edb012cba,2019-08-09 22:07:30.754,41loF2NK40GBUzUBDC1Zrw,NaN,NaN,e
1,0000a0276a0870de648c427edb012cba,2019-08-17 22:09:41.193,41loF2NK40GBUzUBDC1Zrw,NaN,NaN,e
2,0000ae28e29e570fd4cd3d1229b89ec2,2019-08-26 05:48:17.865,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN,d
3,0000be453b034a699e09ba7ec1046fbf,2019-08-31 00:18:02.977,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN,d
4,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-13 14:09:15.755,T1m2A6iwPUemMh5gQZ3FRA,NaN,NaN,a


In [6]:
#기여한 곳 : 1, 기여하지 않은 곳 : 0 을 각각 표시해주기 위한 column인 contributed를 추가하고, 0으로 초기화
df['contributed']=0

In [7]:
#이제 datetime 열의 내용들은 string이 아니라 datetime object가 되어 연산이 빨라짐.
df['datetime'] = df['datetime'].apply(make_datetime)

In [8]:
for index in tqdm_notebook(df.index):
    # partner_index가 conv인 놈을 찾아서
    if(df['partner_index'][index]=='conv'):
        
        standard_adid = df['adid'][index] # 기준이 되는 adid
        standard_datetime = df['datetime'][index] # 기준이 되는 datetime
        
        tmp_index = index-1 # 앵커 인덱스
        df['contributed'][index]=1
        
        while(True): # tmp index에 대한 iteration
            if(standard_adid==df['adid'][tmp_index]): # 동일한 애드아이디
                if((standard_datetime-df['datetime'][tmp_index]).total_seconds()<259200): # 룩백윈도우 이내
                    df['contributed'][tmp_index] = 1 # --> contributed 추가
                else:
                    break; # datetime 조건 break
            else:
                break # adid 조건 break
                
            tmp_index = tmp_index - 1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: 

## 코드 리팩토링

In [9]:
tmpdf = df.loc[:500,:].copy()
tmpdf.head()

,adid,datetime,partner_key,event_name,amount,partner_index,contributed
0,0000a0276a0870de648c427edb012cba,2019-08-09 22:07:30.754,41loF2NK40GBUzUBDC1Zrw,NaN,NaN,e,0
1,0000a0276a0870de648c427edb012cba,2019-08-17 22:09:41.193,41loF2NK40GBUzUBDC1Zrw,NaN,NaN,e,0
2,0000ae28e29e570fd4cd3d1229b89ec2,2019-08-26 05:48:17.865,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN,d,0
3,0000be453b034a699e09ba7ec1046fbf,2019-08-31 00:18:02.977,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN,d,0
4,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-13 14:09:15.755,T1m2A6iwPUemMh5gQZ3FRA,NaN,NaN,a,0


In [10]:
def in_window(seconds, start, end):
    return (start-end).total_seconds() < seconds

In [28]:
for _, group in tmpdf.groupby('adid'):
    events = group['datetime'].loc[group['partner_index']=='conv'] 
    
    if events.empty: # 아예 conv 경우가 없는 group은 empty series 반환
        continue # 해당 adid의 group 건너뛰기
        
    pd.cut(group.datetime, events)
    break

TypeError: category, object, and string subtypes are not supported for IntervalIndex

In [ ]:
#여기서 코드 상 임시저장을 해야됨 ㅠ
df.to_csv('lookback_data_marked_one.csv',encoding = 'utf-8', index = False)

In [ ]:
####여기서부터 null로 끝나는 path들을 만드는 과정임####

In [ ]:
#0만 남길 테이블을 만들어보자
df_zero = pd.read_csv('lookback_data_marked_one.csv')

In [ ]:
#1인 행들을 제거하는 코드임
indexNames = df_zero[df_zero['contributed'] == 1 ].index
df_zero.drop(indexNames , inplace=True)

In [ ]:
df_zero.head(50)

In [ ]:
#인덱스를 다시 매겨보자
df_zero = df_zero.reset_index(drop=True)

In [ ]:
#일단 adid_same을 만들어보자
df_zero['adid2']=df_zero['adid'].shift(1)
df_zero['adid_same']=df_zero['adid']==df_zero['adid2']

In [ ]:
#새로 불러왔으니 다시 datetime을 string->datetime으로 타입변경
df_zero['datetime'] = df_zero['datetime'].apply(make_datetime)

In [ ]:
#zip()으로 range(len(df)), adid_same, datetime들만 존재하는 튜플을 만들어보자!
#zip은 튜플로 구성된 iterator를 반환해준다.
zipped_zero = zip(range(len(df_zero)),df_zero['adid_same'],df_zero['datetime'], df_zero['partner_index'])

In [ ]:
#새로운 dataframe을 만들자. 얘는 나중에 null로 끝날 패스들의 행을 모두 갖게 됨
columns = ['path', 'conversion','null', 'amount']
result_zero = pd.DataFrame(columns=columns)

In [ ]:
#일단 session에 포함될 수 있는 이벤트의 종류는 총 몇개인가? 확인 차 넣은 코드
df_zero['partner_index'].unique()

In [ ]:
#session이 언제 나뉘어야 할까?
# 1. adid가 전행과 다를 때
# 2. 기준점으로부터 시간이 날짜수*86400초를 넘었을 때

#이 null_paths에는 생성된 모든 패스들이 생성된 순서대로 담기게 된다. 이후 result_zero의 'path' 열에 바로 붙일 것임!
null_paths = []

#얘들은 아래 코드를 돌리기 위해 외부에 설정해놓은 변수
start = True
standard_datetime = ""
path_list = []
for row in tqdm_notebook(list(zipped_zero)):
    if(start):#아예 첫 행
        standard_datetime = row[2]
        path_list.append(row[3])
        start = False
    else:#그 외의 행
        #adid가 같은 경우
        if(row[1]):
            #룩백 윈도우를 넘은 경우
            if((row[2]-standard_datetime).total_seconds()>=259200):
                path = ''.join(path_list)
                path_list = []
                path_list.append(row[3])
                null_paths.append(path)
                standard_datetime = row[2]
            else:
                path_list.append(' > ')
                path_list.append(row[3])
        #adid가 바뀐 경우 : 당연히 세션이 새로 시작되어야 함
        else:
            path = ''.join(path_list)
            path_list = []
            path_list.append(row[3])
            null_paths.append(path)
            standard_datetime = row[2]

In [ ]:
result_zero['path']=null_paths
result_zero['conversion']=0
result_zero['null']=1
result_zero['amount']=0

In [ ]:
####여기서부터 conversion으로 끝나는 path들을 만드는 과정임####

In [ ]:
df_one = pd.read_csv('lookback_data_marked_one.csv')

In [ ]:
#0인 행들을 제거하는 코드임
indexNames = df_one[df_one['contributed'] == 0 ].index
df_one.drop(indexNames , inplace=True)

In [ ]:
#df를 뒤집어보자
df_one = df_one.iloc[::-1]

In [ ]:
#인덱스를 다시 매겨보자
df_one = df_one.reset_index(drop=True)

In [ ]:
#adid_same을 만들어보자.
df_one['adid2']=df_one['adid'].shift(1)
df_one['adid_same']=df_one['adid']==df_one['adid2']

In [ ]:
#이제 datetime 열의 요소들은 string이 아니라 실제 datetime
df_one['datetime'] = df_one['datetime'].apply(make_datetime)

In [ ]:
#zip()으로 range(len(df)), adid_same, datetime, partner_index들만 존재하는 튜플을 만들어보자!
#zip은 튜플로 구성된 iterator를 반환해준다.
zipped_one = zip(range(len(df_one)),df_one['adid_same'],df_one['datetime'], df_one['partner_index'])

In [ ]:
#새로운 dataframe을 만들자. columns는 위와 동일하니 그대로.
result_one = pd.DataFrame(columns=columns)

In [ ]:
#일단 session에 포함될 수 있는 이벤트의 종류는?
df_one['partner_index'].unique()

In [ ]:
conversion_paths = []
amounts = []

In [ ]:
#dict를 안 쓰는 버전으로 해보자
#session이 언제 나뉘어야 할까?
# 1. adid가 전행과 다를 때
# 2. 기준점으로부터 시간이 날짜수*86400초를 넘었을 때

start = True
standard_datetime = ""
path_list = []
limit = len(df_one)
for row in tqdm_notebook(list(zipped_one)):
    if(row[3]=='conv'):
        standard_datetime = row[2]
        last = True
        tmp_index = row[0]+1
        if(tmp_index==limit):
            break
        while(True):
            if(df_one['adid_same'][tmp_index]):
                if((standard_datetime-df_one['datetime'][tmp_index]).total_seconds()<259200):
                    if(df_one['partner_index'][tmp_index]!='conv'):
                        if(last):
                            path_list.append(df_one['partner_index'][tmp_index])
                            last = False
                        else:
                            path_list.insert(0, ' > ')
                            path_list.insert(0, df_one['partner_index'][tmp_index])
                    tmp_index = tmp_index + 1
                    if(tmp_index == limit):
                        break
                    continue
                else:
                    break
            else:
                break
        if(len(path_list)>0):
            conversion_paths.append(''.join(path_list))
            amounts.append(df_one['amount'][row[0]])
            path_list = []

In [ ]:
result_one['path']=conversion_paths
result_one['amount']=amounts
result_one['conversion']=1
result_one['null']=0

In [ ]:
#이제 result_zero와 result_one을 합치자.
result_complete = pd.concat([result_zero, result_one], ignore_index=True)

In [ ]:
result_complete.tail(50)

In [ ]:
result_complete.to_csv('lookback_path_complete.csv', encoding = 'utf-8', index = False)